In [1]:
import pandas as pd
import geopandas as gpd

In [150]:
!pip install geopy
import geopy as gpy

  Using cached geopy-2.3.0-py3-none-any.whl (119 kB)
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)


In [159]:
from geopy.geocoders import Nominatim

In [15]:
statesLatLon = pd.read_csv("data/statelatlong.csv")

In [154]:
CommunityCenters = pd.read_csv("data/LGBTCommCenters.csv")

0      290 West Nine Mile Road
1          565 Boylston Street
2             14 Beacon Street
3        53-990 Enterprise Way
4      3131 Santa Anita Avenue
                ...           
431                       9999
432                       9999
433    123 West Madison Street
434       218 West Main Street
435                        NaN
Name: Street Address, Length: 436, dtype: object

In [7]:
antiLGBTBills = pd.read_csv("data/us-anti-lgbtq-legislation-2023.csv", encoding = 'latin1')

,State,Bill,ISSUE,Status,Status Detail
0,Alaska,HB 27,Schools & Education,Advancing,"Referred to committee, 01/19/2023"
1,Alaska,HB 105,Schools & Education,Advancing,"First read and referred to committee, 03/08/2023"
2,Alaska,SB 96,Schools & Education,Advancing,"First read and referred to committee, 03/08/2023"
3,Arizona,SB 1028,Free Speech & Expression,Advancing,"Passed Senate; House hearing, 03/29/2023"
4,Arizona,SB 1026,Free Speech & Expression,Advancing,"Passed Senate; House second read, 03/09/2023"
...,...,...,...,...,...
430,Wyoming,SF 111,Healthcare,Defeated,Passed Senate; House did not consider for intr...
431,Wyoming,SF 117,Schools & Education,Defeated,Passed Senate; House did not consider for intr...
432,Wyoming,HB 262,Civil Rights,Defeated,"Died in committee, 02/07/2023"
433,Wyoming,SF 133,Schools & Education,Passed into Law,"Became act without Governors signature, 03/17..."


In [33]:
antiLGBTBills_latlon = antiLGBTBills.join(statesLatLon.set_index('City'), on='State', lsuffix='_left', rsuffix='_right')
antiLGBTBills_latlon['State_left'][0]

'Alaska'

In [17]:
antiLGBTBills_gdf = gpd.GeoDataFrame(
    antiLGBTBills_latlon, geometry=gpd.points_from_xy(antiLGBTBills_latlon.Longitude, antiLGBTBills_latlon.Latitude))
antiLGBTBills_gdf.to_file('data/antiLGBTBills.geojson', driver='GeoJSON') 

In [18]:
LGBTPop = pd.read_csv("data/LGBTPop.csv")
LGBTPop_latlon = LGBTPop.join(statesLatLon.set_index('State'), on='State', lsuffix='_left', rsuffix='_right')
LGBTPop_gdf = gpd.GeoDataFrame(
    LGBTPop_latlon, geometry=gpd.points_from_xy(LGBTPop_latlon.Longitude, LGBTPop_latlon.Latitude))
LGBTPop_gdf.to_file('data/LGBTPop.geojson', driver='GeoJSON') 

In [145]:
antiLGBTBills_byState = pd.DataFrame(columns = ["State", "Bills", "Count", "Latitude", "Longitude"])
antiLGBTBills_byState['State'] = ['Alaska', 'Arizona']
antiLGBTBills_byState['Bills'] = ['', '']
for i in range(42):
    antiLGBTBills_byState.loc[i+2] = str("")
antiLGBTBills_byState



,State,Bills,Count,Latitude,Longitude
0,Alaska,,NaN,NaN,NaN
1,Arizona,,NaN,NaN,NaN
2,,,,,
3,,,,,
4,,,,,
5,,,,,
6,,,,,
7,,,,,
8,,,,,
9,,,,,


In [146]:
count = 1
for i in antiLGBTBills_latlon['State_left']:
    if i == antiLGBTBills_byState['State'][count-1]:
        count = count - 1
    else:
        antiLGBTBills_byState.loc[count, 'State'] = i
    count = count + 1
antiLGBTBills_byState

,State,Bills,Count,Latitude,Longitude
0,Alaska,,NaN,NaN,NaN
1,Arizona,,NaN,NaN,NaN
2,Arkansas,,,,
3,California,,,,
4,Colorado,,,,
5,Connecticut,,,,
6,Florida,,,,
7,Georgia,,,,
8,Hawaii,,,,
9,Idaho,,,,


In [147]:
latlon_count = 0
bystate_count = 0
bill_count = 0
for i in antiLGBTBills_latlon['Bill']:
    if latlon_count < 434:
        if antiLGBTBills_latlon["State_left"][latlon_count] == antiLGBTBills_byState['State'][bystate_count]:
            antiLGBTBills_byState['Bills'][bystate_count] += i + ", "
            bill_count += 1
            if antiLGBTBills_latlon["State_left"][latlon_count+1] == antiLGBTBills_byState['State'][bystate_count]:
                bystate_count -= 1
            else: 
                antiLGBTBills_byState['Count'][bystate_count] = bill_count
                antiLGBTBills_byState['Latitude'][bystate_count] = antiLGBTBills_latlon["Latitude"][latlon_count-2]
                antiLGBTBills_byState['Longitude'][bystate_count] = antiLGBTBills_latlon["Longitude"][latlon_count-2]
                bill_count = 0
        bystate_count += 1
        latlon_count += 1
        
antiLGBTBills_byState['Count'][43] = bill_count
antiLGBTBills_byState['Latitude'][43] = antiLGBTBills_latlon["Latitude"][latlon_count-1]
antiLGBTBills_byState['Longitude'][43] = antiLGBTBills_latlon["Longitude"][latlon_count-1]
antiLGBTBills_byState


,State,Bills,Count,Latitude,Longitude
0,Alaska,"HB 27, HB 105, SB 96,",3,61.302501,-158.77502
1,Arizona,"SB 1028, SB 1026, SB 1001, SB 1030, SB 1040, S...",11,34.168219,-111.930907
2,Arkansas,"HB 1156, SB 43, SB 125, SB 199, SB 270, HB 146...",9,34.751928,-92.131378
3,California,"AB 1314,",1,34.751928,-92.131378
4,Colorado,"HB 23-1098,",1,34.751928,-92.131378
5,Connecticut,"HB 6213, SB 468,",2,38.997934,-105.550567
6,Florida,"HB 991, HB 1069, HB 1223, SB 1320, HB 999, SB ...",10,27.975728,-81.833017
7,Georgia,"SB 88, SB 141, SB 140, SB 180, HB 653,",5,32.678125,-83.222976
8,Hawaii,"HB 509, HB 508, HB 891, SB 1429,",4,20.46,-157.505
9,Idaho,"S 1003, SB 1016, HB 71, SB 1100, HB 63, SB 107...",7,45.494576,-114.14243


In [148]:
antiLGBTBills_byState_gdf = gpd.GeoDataFrame(
    antiLGBTBills_byState, geometry=gpd.points_from_xy(antiLGBTBills_byState.Longitude, antiLGBTBills_byState.Latitude))
antiLGBTBills_byState_gdf.to_file('data/antiLGBTBills_byState.geojson', driver='GeoJSON') 

In [ ]:
CommunityCenters["Latitidue"] = ""
CommunityCenters["Longitude"] = ""

In [227]:
locator = Nominatim(user_agent="myGeocoder")
print(CommunityCenters["Street Address"][i] + ", " + CommunityCenters["City"][i] + ", " + CommunityCenters["State"][i])
count = 0
for i in range(433, 435):
    location = locator.geocode(CommunityCenters["Street Address"][i] + ", " + CommunityCenters["City"][i] + ", " + CommunityCenters["State"][i] + " ")
    print(locator.geocode(CommunityCenters["Street Address"][i] + ", " + CommunityCenters["City"][i] + ", " + CommunityCenters["State"][i]))
    try:
        CommunityCenters["Latitidue"][i] = location.latitude
        CommunityCenters["Longitude"][i] = location.longitude
    except:
        CommunityCenters["Latitidue"][i] = "NaN"
        CommunityCenters["Longitude"][i] = "NaN"
    count += 1
    print(count)
CommunityCenters[:][360:435]

9999, Brooklyn, New York
123, West Madison Street, Loop, Chicago, Cook County, Illinois, 60603, United States
1


/tmp/ipykernel_648/3452628465.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CommunityCenters["Latitidue"][i] = location.latitude
/tmp/ipykernel_648/3452628465.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CommunityCenters["Longitude"][i] = location.longitude


218, West Main Street, Farmington, San Juan County, New Mexico, 87401, United States
2


,ID,Center Name,ServeLesbian,ServeGay,ServeBisexual,ServeTransgender,GeoCoverage,WebPresence,WebAddress,NGOStatus,...,CommunityCenter,CommHealthCenter,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Latitidue,Longitude
360,361.0,Back Country Bettys,1.0,0.0,0.0,0.0,2.0,1.0,https://sites.google.com/site/backcountrybetty...,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,40.550853,-105.066808
361,362.0,Equality Virginia,1.0,1.0,1.0,1.0,1.0,1.0,http://www.equalityvirginia.org/,1.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,38.126849,-76.606362
362,363.0,Hampton Roads Pride,1.0,1.0,1.0,1.0,2.0,1.0,http://www.hamptonroadspride.org/,1.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,52.447765,0.754011
363,364.0,Northern Virginia Pride,1.0,1.0,1.0,1.0,2.0,1.0,http://www.novapride.org/,1.0,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,365.0,Equalize,1.0,1.0,1.0,1.0,2.0,1.0,http://equalizeit.com/,1.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,45.54007,-122.628226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,431.0,"Two Spirit Health Services, Inc.",1.0,1.0,1.0,1.0,2.0,1.0,http://www.twospirithealth.org/,1.0,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,28.538524,-81.377391
431,432.0,Texas Two Spirit Society,1.0,1.0,1.0,1.0,1.0,1.0,http://texastwospirits.com/,1.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,32.191289,-96.87842
432,433.0,Armenian Gay & Lesbian Association of New York,1.0,1.0,1.0,1.0,1.0,1.0,http://aglany.org/,1.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,,
433,434.0,Reconciling Ministries Network,1.0,1.0,1.0,1.0,0.0,1.0,http://www.rmnetwork.org/newrmn/,1.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,41.881737,-87.631592


In [243]:
for i in range(435):
    if CommunityCenters["Latitude"][i] == '':
        CommunityCenters["Latitude"][i] = float(-9999)
        CommunityCenters["Longitude"][i] = float(-9999)
        

In [247]:
CommunityCenters = CommunityCenters.rename(columns={"Latitidue": "Latitude"})

for i in range(436):
    try:
        CommunityCenters["Latitude"][i] = float(CommunityCenters["Latitude"][i])
        CommunityCenters["Longitude"][i] = float(CommunityCenters["Longitude"][i])
    except:
        CommunityCenters["Latitude"][i] = float(-999999)
        CommunityCenters["Longitude"][i] = float(-999999)
        
CommunityCenters

CommunityCenters_gdf = gpd.GeoDataFrame(
   CommunityCenters, geometry=gpd.points_from_xy(CommunityCenters.Longitude, CommunityCenters.Latitude))
CommunityCenters_gdf.to_file('data/CommunityCenters.geojson', driver='GeoJSON') 

/tmp/ipykernel_648/2313234519.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CommunityCenters["Latitude"][i] = float(CommunityCenters["Latitude"][i])
/tmp/ipykernel_648/2313234519.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CommunityCenters["Longitude"][i] = float(CommunityCenters["Longitude"][i])


In [248]:
antiLGBTBills_byState_gdf.to_file('data/antiLGBTBills_byState.geojson', driver='GeoJSON')
LGBTPop_gdf.to_file('data/LGBTPop.geojson', driver='GeoJSON')
antiLGBTBills_gdf.to_file('data/AntiLGBTBills.geojson', driver='GeoJSON')

In [249]:
antiLGBTBills_gdf

,State_left,Bill,ISSUE,Status,Status Detail,State_right,Latitude,Longitude,geometry
0,Alaska,HB 27,Schools & Education,Advancing,"Referred to committee, 01/19/2023",AK,61.302501,-158.775020,POINT (-158.77502 61.30250)
1,Alaska,HB 105,Schools & Education,Advancing,"First read and referred to committee, 03/08/2023",AK,61.302501,-158.775020,POINT (-158.77502 61.30250)
2,Alaska,SB 96,Schools & Education,Advancing,"First read and referred to committee, 03/08/2023",AK,61.302501,-158.775020,POINT (-158.77502 61.30250)
3,Arizona,SB 1028,Free Speech & Expression,Advancing,"Passed Senate; House hearing, 03/29/2023",AZ,34.168219,-111.930907,POINT (-111.93091 34.16822)
4,Arizona,SB 1026,Free Speech & Expression,Advancing,"Passed Senate; House second read, 03/09/2023",AZ,34.168219,-111.930907,POINT (-111.93091 34.16822)
...,...,...,...,...,...,...,...,...,...
430,Wyoming,SF 111,Healthcare,Defeated,Passed Senate; House did not consider for intr...,WY,43.000325,-107.554567,POINT (-107.55457 43.00032)
431,Wyoming,SF 117,Schools & Education,Defeated,Passed Senate; House did not consider for intr...,WY,43.000325,-107.554567,POINT (-107.55457 43.00032)
432,Wyoming,HB 262,Civil Rights,Defeated,"Died in committee, 02/07/2023",WY,43.000325,-107.554567,POINT (-107.55457 43.00032)
433,Wyoming,SF 133,Schools & Education,Passed into Law,"Became act without Governors signature, 03/17...",WY,43.000325,-107.554567,POINT (-107.55457 43.00032)


In [250]:
LGBTPop_gdf

,State,Trans Pop,Gay Pop,Bi Pop,Latitude,Longitude,City,geometry
0,AL,"99,421","107,028","118,661",32.601011,-86.680736,Alabama,POINT (-86.68074 32.60101)
1,AK,"5,731","16,399","27,175",61.302501,-158.775020,Alaska,POINT (-158.77502 61.30250)
2,AZ,"44,005","317,917","288,714",34.168219,-111.930907,Arizona,POINT (-111.93091 34.16822)
3,AR,"22,089","49,159","66,813",34.751928,-92.131378,Arkansas,POINT (-92.13138 34.75193)
4,CA,"129,080","1,313,001","1,313,890",37.271875,-119.270415,California,POINT (-119.27042 37.27187)
5,CO,"19,512","215,899","261,377",38.997934,-105.550567,Colorado,POINT (-105.55057 38.99793)
6,CT,"63,653","121,866","146,612",41.518783,-72.757507,Connecticut,POINT (-72.75751 41.51878)
7,DE,"7,657","21,773","30,873",39.145251,-75.418921,Delaware,POINT (-75.41892 39.14525)
8,DC,178,"59,742","22,079",38.899349,-77.014567,District of Columbia,POINT (-77.01457 38.89935)
9,FL,"164,974","572,189","595,017",27.975728,-81.833017,Florida,POINT (-81.83302 27.97573)
